In [ ]:
# XGBOOST V3.1
!pip install -q pandas numpy scikit-learn matplotlib xgboost scikit-optimize category_encoders yfinance joblib
!pip install -q --upgrade yfinance

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBRegressor
from skopt import BayesSearchCV
import joblib
import yfinance as yf
import matplotlib.pyplot as plt

# Column Names
FUEL_COL = 'Fuel price(AED)'
OIL_COL = 'Crude Oil Barrel Price (USD)'
GOLD_COL = 'Gold Prices (AED)'

# Real-time Prices API
def get_crude_price():
    crude = yf.Ticker("CL=F")
    data = crude.history(period="5d")
    if data.empty or 'Close' not in data.columns:
        raise ValueError("Failed to retrieve crude oil data from Yahoo Finance.")
    return data['Close'].iloc[0]

def get_latest_gold_price():
    gold = yf.Ticker("GC=F")
    data = gold.history(period="5d", interval="1m")
    if data.empty or 'Close' not in data.columns:
        raise ValueError("Failed to retrieve gold price data from Yahoo Finance.")
    return data['Close'].iloc[-1]

coprice = get_crude_price()
gold_price = get_latest_gold_price()

# Load Excel Dataset
df = pd.read_excel("FuelData.xlsx", index_col=0, parse_dates=True)
df.sort_index(inplace=True)

# Safe Time-Series Feature Creation
def add_time_series_features_safe(df, max_lag=12):
    df = df.copy()
    n_rows = len(df)
    
    # Adjust max_lag if dataset is small
    max_lag = min(max_lag, n_rows - 1)
    
    # Create lag features
    for lag in range(1, max_lag + 1):
        df[f'fuel_lag_{lag}'] = df[FUEL_COL].shift(lag)
        df[f'gold_lag_{lag}'] = df[GOLD_COL].shift(lag)
        df[f'oil_lag_{lag}'] = df[OIL_COL].shift(lag)
    
    # Rolling statistics (window cannot exceed available rows)
    window = min(3, n_rows)
    df['fuel_rolling_avg'] = df[FUEL_COL].rolling(window).mean()
    df['fuel_rolling_std'] = df[FUEL_COL].rolling(window).std()
    df['gold_rolling_avg'] = df[GOLD_COL].rolling(window).mean()
    df['gold_rolling_std'] = df[GOLD_COL].rolling(window).std()
    df['oil_rolling_avg'] = df[OIL_COL].rolling(window).mean()
    df['oil_rolling_std'] = df[OIL_COL].rolling(window).std()
    
    # Percentage change & stability
    df['fuel_pct_change_1'] = df[FUEL_COL].pct_change()
    df['is_stable_recently'] = (df['fuel_pct_change_1'].abs() < 0.01).astype(int)
    
    # Only drop rows where all lags are NaN
    lag_cols = [f'fuel_lag_{i}' for i in range(1, max_lag+1)]
    df = df.dropna(subset=lag_cols)
    
    return df

df = add_time_series_features_safe(df)

# Prepare Features and Target
X = df.drop(columns=[FUEL_COL, 'fuel_pct_change_1'])
y = df[FUEL_COL]

# Safe Train-Test Split
n_splits = min(5, len(X)-1)
tscv = TimeSeriesSplit(n_splits=n_splits)

splits = list(tscv.split(X))
train_index, test_index = splits[-1]

X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Modeling Pipeline
pipe = Pipeline([
    ('encoder', TargetEncoder()),
    ('regressor', XGBRegressor(random_state=8, enable_categorical=True))
])

param_grid = {
    'regressor__n_estimators': (100, 1500),
    'regressor__learning_rate': (0.01, 0.9, 'log-uniform'),
    'regressor__max_depth': (1, 9),
    'regressor__subsample': (0.4, 1.0),
    'regressor__colsample_bytree': (0.4, 1.0)
}

opt = BayesSearchCV(
    pipe,
    param_grid,
    n_iter=50,
    cv=tscv,
    scoring="r2",
    random_state=8,
    n_jobs=-1
)

opt.fit(X_train, y_train)
best_model = opt.best_estimator_

# Evaluate Model
y_pred = best_model.predict(X_test)
print("\n=== Model Performance ===")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")

# Forecast Next Month
def forecast_next_month(df, model, current_price):
    last = df.iloc[-1].copy()
    features = last.drop(FUEL_COL).to_dict()

    # Only use lags that exist
    lag_cols = [col for col in features if 'fuel_lag_' in col]
    if len(lag_cols) >= 2:
        features.update({
            'fuel_lag_1': current_price,
            'fuel_lag_2': last['fuel_lag_1'],
            'fuel_rolling_avg': np.mean([current_price, last['fuel_lag_1'], last['fuel_lag_2']]),
            'fuel_rolling_std': np.std([current_price, last['fuel_lag_1'], last['fuel_lag_2']]),
            'fuel_pct_change_1': (current_price - last['fuel_lag_1']) / last['fuel_lag_1'],
            'is_stable_recently': int(abs(current_price - last['fuel_lag_1']) < 0.01)
        })

    next_input = pd.DataFrame([features])
    expected_columns = model.named_steps['regressor'].get_booster().feature_names
    next_input = next_input[expected_columns]

    pred = model.predict(next_input)[0]

    # Limit unlikely volatility
    max_expected_change = 0.09
    if abs(pred - current_price) > max_expected_change:
        print("Prediction adjusted due to unlikely volatility.")
        pred = current_price + max_expected_change if pred > current_price else current_price - max_expected_change

    return pred

this_month_price = df[FUEL_COL].iloc[-1]
next_month_pred = forecast_next_month(df, best_model, this_month_price)

print(f"\nLast recorded fuel price: {this_month_price:.2f} AED")
print(f"Forecast for next month: {next_month_pred:.2f} AED")
print(f"Change: {next_month_pred - this_month_price:.2f} AED")

# Analyze Trend
def analyze_trend(current, predicted):
    change = predicted - current
    pct_change = (abs(change) / current) * 100
    if abs(change) < 0.01:
        return "Price expected to remain stable"
    direction = "increase" if change > 0 else "decrease"
    return f"Expected {direction} of {abs(change):.2f} AED ({pct_change:.1f}%)"

print("\n" + analyze_trend(this_month_price, next_month_pred))